# 🧠 Agent Short-Term Memory with MongoDB

This notebook demonstrates how to implement **short-term memory** for the ConnectBest chat agent using MongoDB.

## Memory Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                     Memory Collection                        │
├─────────────────────────────────────────────────────────────┤
│  {                                                           │
│    "_id": ObjectId,                                          │
│    "session_id": "user_123_1701388800",                     │
│    "user_id": "user_123",                                    │
│    "messages": [                                             │
│      {"role": "user", "content": "...", "ts": ...},         │
│      {"role": "assistant", "content": "...", "ts": ...}     │
│    ],                                                        │
│    "context": {...},  # Extracted entities, preferences     │
│    "created_at": datetime,                                   │
│    "updated_at": datetime,                                   │
│    "expires_at": datetime  # TTL for auto-cleanup           │
│  }                                                           │
└─────────────────────────────────────────────────────────────┘
```

## Features
1. **Session-based memory** - Conversations grouped by session
2. **TTL expiration** - Auto-cleanup of old sessions (e.g., 24 hours)
3. **Context extraction** - Remember user preferences and entities
4. **Sliding window** - Keep last N messages to manage token limits

In [1]:
# Install required packages
import subprocess
subprocess.run(["pip", "install", "-q", "pymongo", "python-dotenv"], check=True)
print("✅ Packages installed!")

✅ Packages installed!


## 1. Connect to MongoDB

In [4]:
import os
import sys
from datetime import datetime, timedelta
from typing import Optional, List, Dict, Any
from pymongo import MongoClient, ASCENDING, DESCENDING
from pymongo.server_api import ServerApi
from pymongo.collection import Collection
from dotenv import load_dotenv
import json

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath(".."))

# Load environment variables from app's .env
load_dotenv("../app/api/tools/.env")

# Connect to MongoDB using MONGO_URI (same as the app uses)
MONGO_URI = os.getenv("MONGO_URI")
DATABASE_NAME = os.getenv("MONGO_DATABASE", "connectbest_chat")

if not MONGO_URI:
    raise ValueError("❌ MONGO_URI not found in .env file! Please set it.")

client = MongoClient(MONGO_URI, server_api=ServerApi('1'))
db = client[DATABASE_NAME]

# Test connection
client.admin.command('ping')
print(f"✅ Connected to MongoDB: {DATABASE_NAME}")
print(f"📚 Collections: {db.list_collection_names()}")

✅ Connected to MongoDB: connectbest_chat
📚 Collections: ['channel_members', 'messages', 'message_embeddings', 'reactions', 'files', 'channels', 'threads', 'message_files', 'users']


## 2. Define the Memory Manager Class

This class handles all memory operations:
- **Session management** - Create/retrieve sessions by user ID
- **Message storage** - Store user/assistant messages with timestamps  
- **Context extraction** - Remember entities, preferences mentioned
- **Window management** - Keep last N messages to manage token limits
- **TTL expiration** - Auto-cleanup old sessions

In [5]:
class AgentMemory:
    """
    Short-term memory manager for the ConnectBest agent.
    
    Stores conversation history in MongoDB with:
    - Session-based grouping (conversations expire after TTL)
    - Sliding window (last N messages)
    - Context extraction (entities, preferences)
    - TTL-based expiration (auto-cleanup)
    """
    
    COLLECTION_NAME = "agent_memory"
    DEFAULT_TTL_HOURS = 24  # Sessions expire after 24 hours
    DEFAULT_MAX_MESSAGES = 20  # Keep last 20 messages per session
    
    def __init__(
        self, 
        db, 
        ttl_hours: int = DEFAULT_TTL_HOURS,
        max_messages: int = DEFAULT_MAX_MESSAGES
    ):
        self.db = db
        self.collection: Collection = db[self.COLLECTION_NAME]
        self.ttl_hours = ttl_hours
        self.max_messages = max_messages
        
        # Ensure indexes exist
        self._setup_indexes()
    
    def _setup_indexes(self):
        """Create necessary indexes for efficient queries."""
        # Index for fast user lookup
        self.collection.create_index([("user_id", ASCENDING)])
        
        # Index for session lookup
        self.collection.create_index([("session_id", ASCENDING)], unique=True)
        
        # TTL index for auto-expiration
        # MongoDB will automatically delete documents when expires_at < now
        self.collection.create_index(
            [("expires_at", ASCENDING)],
            expireAfterSeconds=0  # Delete when expires_at time is reached
        )
        
        print("✅ Memory indexes created")
    
    def _generate_session_id(self, user_id: str) -> str:
        """Generate a unique session ID for a user."""
        # Session ID includes timestamp to create new sessions over time
        timestamp = int(datetime.utcnow().timestamp())
        return f"{user_id}_{timestamp}"
    
    def get_or_create_session(self, user_id: str) -> Dict[str, Any]:
        """
        Get the active session for a user, or create a new one.
        
        A session is considered "active" if:
        - It exists and hasn't expired
        - It was updated within the TTL window
        """
        now = datetime.utcnow()
        cutoff = now - timedelta(hours=self.ttl_hours)
        
        # Find active session
        session = self.collection.find_one({
            "user_id": user_id,
            "updated_at": {"$gt": cutoff}
        }, sort=[("updated_at", DESCENDING)])
        
        if session:
            return session
        
        # Create new session
        session_id = self._generate_session_id(user_id)
        new_session = {
            "session_id": session_id,
            "user_id": user_id,
            "messages": [],
            "context": {
                "entities": [],      # Named entities mentioned
                "preferences": {},   # User preferences
                "topics": [],        # Topics discussed
            },
            "created_at": now,
            "updated_at": now,
            "expires_at": now + timedelta(hours=self.ttl_hours)
        }
        
        self.collection.insert_one(new_session)
        print(f"📝 Created new session: {session_id}")
        return new_session
    
    def add_message(
        self, 
        user_id: str, 
        role: str, 
        content: str,
        metadata: Optional[Dict] = None
    ) -> Dict[str, Any]:
        """
        Add a message to the user's active session.
        
        Args:
            user_id: The user's ID
            role: "user" or "assistant"
            content: The message content
            metadata: Optional metadata (tool calls, etc.)
        
        Returns:
            The updated session document
        """
        now = datetime.utcnow()
        
        message = {
            "role": role,
            "content": content,
            "timestamp": now,
            "metadata": metadata or {}
        }
        
        # Get or create session
        session = self.get_or_create_session(user_id)
        
        # Update session with new message
        result = self.collection.update_one(
            {"session_id": session["session_id"]},
            {
                "$push": {
                    "messages": {
                        "$each": [message],
                        "$slice": -self.max_messages  # Keep only last N messages
                    }
                },
                "$set": {
                    "updated_at": now,
                    "expires_at": now + timedelta(hours=self.ttl_hours)  # Extend TTL
                }
            }
        )
        
        return self.get_session(session["session_id"])
    
    def get_session(self, session_id: str) -> Optional[Dict[str, Any]]:
        """Get a session by ID."""
        return self.collection.find_one({"session_id": session_id})
    
    def get_conversation_history(
        self, 
        user_id: str, 
        limit: Optional[int] = None
    ) -> List[Dict[str, Any]]:
        """
        Get the conversation history for a user.
        
        Args:
            user_id: The user's ID
            limit: Optional limit on messages to return
        
        Returns:
            List of messages in chronological order
        """
        session = self.get_or_create_session(user_id)
        messages = session.get("messages", [])
        
        if limit:
            messages = messages[-limit:]
        
        return messages
    
    def get_formatted_history(
        self, 
        user_id: str, 
        limit: Optional[int] = None
    ) -> str:
        """
        Get conversation history formatted for LLM context.
        
        Returns a string like:
        User: What channels are there?
        Assistant: I found 5 channels...
        User: Tell me about #general
        """
        messages = self.get_conversation_history(user_id, limit)
        
        formatted = []
        for msg in messages:
            role = "User" if msg["role"] == "user" else "Assistant"
            formatted.append(f"{role}: {msg['content']}")
        
        return "\n".join(formatted)
    
    def update_context(
        self, 
        user_id: str, 
        entities: Optional[List[str]] = None,
        preferences: Optional[Dict] = None,
        topics: Optional[List[str]] = None
    ):
        """
        Update the context for a user's session.
        
        This stores extracted information like:
        - Entities: "Alice", "Project X", "#general"
        - Preferences: {"timezone": "PST", "preferred_channel": "general"}
        - Topics: ["meeting scheduling", "project updates"]
        """
        session = self.get_or_create_session(user_id)
        
        update = {"$set": {"updated_at": datetime.utcnow()}}
        
        if entities:
            update["$addToSet"] = {"context.entities": {"$each": entities}}
        
        if preferences:
            for key, value in preferences.items():
                update.setdefault("$set", {})
                update["$set"][f"context.preferences.{key}"] = value
        
        if topics:
            update.setdefault("$addToSet", {})
            update["$addToSet"]["context.topics"] = {"$each": topics}
        
        self.collection.update_one(
            {"session_id": session["session_id"]},
            update
        )
    
    def get_context(self, user_id: str) -> Dict[str, Any]:
        """Get the stored context for a user."""
        session = self.get_or_create_session(user_id)
        return session.get("context", {})
    
    def clear_session(self, user_id: str) -> bool:
        """Clear/delete a user's current session."""
        result = self.collection.delete_many({"user_id": user_id})
        return result.deleted_count > 0
    
    def get_all_sessions(self, user_id: str) -> List[Dict[str, Any]]:
        """Get all sessions for a user (for debugging)."""
        return list(self.collection.find(
            {"user_id": user_id},
            sort=[("created_at", DESCENDING)]
        ))
    
    def get_stats(self) -> Dict[str, Any]:
        """Get memory statistics."""
        total_sessions = self.collection.count_documents({})
        active_sessions = self.collection.count_documents({
            "expires_at": {"$gt": datetime.utcnow()}
        })
        
        return {
            "total_sessions": total_sessions,
            "active_sessions": active_sessions,
            "expired_sessions": total_sessions - active_sessions,
            "collection": self.COLLECTION_NAME
        }

# Create memory instance
memory = AgentMemory(db)
print(f"📊 Memory Stats: {memory.get_stats()}")

✅ Memory indexes created
📊 Memory Stats: {'total_sessions': 0, 'active_sessions': 0, 'expired_sessions': 0, 'collection': 'agent_memory'}


/tmp/ipykernel_113537/1377085574.py:249: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "expires_at": {"$gt": datetime.utcnow()}


## 3. Demo: Simulate a Conversation

Let's simulate a multi-turn conversation to see how memory works:

In [6]:
# Simulate a user ID (this would come from Slack in production)
TEST_USER_ID = "demo_user_123"

# Clear any existing session for clean demo
memory.clear_session(TEST_USER_ID)
print("🧹 Cleared existing sessions")

# Simulate conversation turn 1
print("\n" + "="*60)
print("📨 Turn 1: User asks about channels")
print("="*60)

memory.add_message(TEST_USER_ID, "user", "What channels are available?")
memory.add_message(
    TEST_USER_ID, 
    "assistant", 
    "I found 5 channels: #general, #engineering, #design, #marketing, and #random.",
    metadata={"tool_used": "list_channels", "channels_found": 5}
)

# Show current history
print("\n📜 Conversation History:")
print(memory.get_formatted_history(TEST_USER_ID))

🧹 Cleared existing sessions

📨 Turn 1: User asks about channels
📝 Created new session: demo_user_123_1764658451


/tmp/ipykernel_113537/1377085574.py:112: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()
/tmp/ipykernel_113537/1377085574.py:61: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()
/tmp/ipykernel_113537/1377085574.py:50: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = int(datetime.utcnow().timestamp())



📜 Conversation History:
User: What channels are available?
Assistant: I found 5 channels: #general, #engineering, #design, #marketing, and #random.


In [7]:
# Simulate conversation turn 2
print("="*60)
print("📨 Turn 2: User asks follow-up about a specific channel")
print("="*60)

memory.add_message(TEST_USER_ID, "user", "Tell me more about #engineering")
memory.add_message(
    TEST_USER_ID, 
    "assistant", 
    "The #engineering channel has 12 members and 234 messages. Recent topics include: API refactoring, CI/CD pipeline, and code reviews. Top contributors: Alice, Bob, and Charlie.",
    metadata={"tool_used": "channel_info", "channel": "engineering"}
)

# Update context with extracted entities
memory.update_context(
    TEST_USER_ID,
    entities=["#engineering", "Alice", "Bob", "Charlie"],
    topics=["engineering", "API refactoring"]
)

print("\n📜 Full Conversation History:")
print(memory.get_formatted_history(TEST_USER_ID))

print("\n🧠 Extracted Context:")
context = memory.get_context(TEST_USER_ID)
print(json.dumps(context, indent=2, default=str))

📨 Turn 2: User asks follow-up about a specific channel


/tmp/ipykernel_113537/1377085574.py:112: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()
/tmp/ipykernel_113537/1377085574.py:61: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()
/tmp/ipykernel_113537/1377085574.py:209: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  update = {"$set": {"updated_at": datetime.utcnow()}}



📜 Full Conversation History:
User: What channels are available?
Assistant: I found 5 channels: #general, #engineering, #design, #marketing, and #random.
User: Tell me more about #engineering
Assistant: The #engineering channel has 12 members and 234 messages. Recent topics include: API refactoring, CI/CD pipeline, and code reviews. Top contributors: Alice, Bob, and Charlie.

🧠 Extracted Context:
{
  "entities": [
    "#engineering",
    "Alice",
    "Bob",
    "Charlie"
  ],
  "preferences": {},
  "topics": [
    "engineering",
    "API refactoring"
  ]
}


In [8]:
# Simulate conversation turn 3 - Reference previous context!
print("="*60)
print("📨 Turn 3: User asks about 'her' (referencing Alice from context)")
print("="*60)

memory.add_message(TEST_USER_ID, "user", "Can you tell me more about her?")

# In production, the agent would use memory to understand "her" = Alice
# Let's simulate that the agent successfully resolved the reference
memory.add_message(
    TEST_USER_ID, 
    "assistant", 
    "Based on our conversation, you're asking about Alice from the #engineering channel. Alice is a Senior Engineer who has been very active in discussions about API refactoring. She posted 45 messages this week.",
    metadata={"tool_used": "user_info", "resolved_reference": "Alice", "used_memory": True}
)

print("\n📜 Full Conversation History:")
print(memory.get_formatted_history(TEST_USER_ID))

print("\n💡 Key Insight: The agent used memory to resolve 'her' → 'Alice'!")

📨 Turn 3: User asks about 'her' (referencing Alice from context)


/tmp/ipykernel_113537/1377085574.py:112: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()
/tmp/ipykernel_113537/1377085574.py:61: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()



📜 Full Conversation History:
User: What channels are available?
Assistant: I found 5 channels: #general, #engineering, #design, #marketing, and #random.
User: Tell me more about #engineering
Assistant: The #engineering channel has 12 members and 234 messages. Recent topics include: API refactoring, CI/CD pipeline, and code reviews. Top contributors: Alice, Bob, and Charlie.
User: Can you tell me more about her?
Assistant: Based on our conversation, you're asking about Alice from the #engineering channel. Alice is a Senior Engineer who has been very active in discussions about API refactoring. She posted 45 messages this week.

💡 Key Insight: The agent used memory to resolve 'her' → 'Alice'!


## 4. View the Raw Session in MongoDB

Let's see what the session looks like in the database:

In [9]:
# View the raw session document
sessions = memory.get_all_sessions(TEST_USER_ID)
print(f"📦 Found {len(sessions)} session(s) for user {TEST_USER_ID}")
print()

if sessions:
    session = sessions[0]
    # Convert to JSON-friendly format
    session_display = {
        "session_id": session["session_id"],
        "user_id": session["user_id"],
        "message_count": len(session["messages"]),
        "context": session["context"],
        "created_at": str(session["created_at"]),
        "updated_at": str(session["updated_at"]),
        "expires_at": str(session["expires_at"]),
    }
    print("📄 Session Document:")
    print(json.dumps(session_display, indent=2))
    
    print("\n📨 Messages with Metadata:")
    for i, msg in enumerate(session["messages"], 1):
        print(f"\n  [{i}] {msg['role'].upper()}:")
        print(f"      Content: {msg['content'][:80]}...")
        if msg.get("metadata"):
            print(f"      Metadata: {msg['metadata']}")

📦 Found 1 session(s) for user demo_user_123

📄 Session Document:
{
  "session_id": "demo_user_123_1764658451",
  "user_id": "demo_user_123",
  "message_count": 6,
  "context": {
    "entities": [
      "#engineering",
      "Alice",
      "Bob",
      "Charlie"
    ],
    "preferences": {},
    "topics": [
      "engineering",
      "API refactoring"
    ]
  },
  "created_at": "2025-12-01 22:54:11.477000",
  "updated_at": "2025-12-01 22:54:55.385000",
  "expires_at": "2025-12-02 22:54:55.385000"
}

📨 Messages with Metadata:

  [1] USER:
      Content: What channels are available?...

  [2] ASSISTANT:
      Content: I found 5 channels: #general, #engineering, #design, #marketing, and #random....
      Metadata: {'tool_used': 'list_channels', 'channels_found': 5}

  [3] USER:
      Content: Tell me more about #engineering...

  [4] ASSISTANT:
      Content: The #engineering channel has 12 members and 234 messages. Recent topics include:...
      Metadata: {'tool_used': 'channel_info', 'c

## 5. Integration with Agent

Here's how to integrate memory into the agent's orchestrator:

In [10]:
def build_prompt_with_memory(user_id: str, current_query: str, memory: AgentMemory) -> str:
    """
    Build a prompt that includes conversation history and context.
    
    This is what the orchestrator would use when calling the LLM.
    """
    # Get conversation history (last 10 messages to manage tokens)
    history = memory.get_formatted_history(user_id, limit=10)
    
    # Get extracted context
    context = memory.get_context(user_id)
    
    # Build the prompt
    prompt_parts = []
    
    # System context
    prompt_parts.append("You are a helpful Slack assistant for ConnectBest.")
    
    # Add extracted context if available
    if context.get("entities"):
        prompt_parts.append(f"\nRelevant entities from this conversation: {', '.join(context['entities'])}")
    if context.get("topics"):
        prompt_parts.append(f"Topics discussed: {', '.join(context['topics'])}")
    
    # Add conversation history
    if history:
        prompt_parts.append(f"\n--- Conversation History ---\n{history}")
    
    # Add current query
    prompt_parts.append(f"\n--- Current Query ---\nUser: {current_query}")
    
    return "\n".join(prompt_parts)

# Demo the prompt building
demo_prompt = build_prompt_with_memory(
    TEST_USER_ID, 
    "Schedule a meeting with her about the API changes",
    memory
)

print("📝 Generated Prompt for LLM:")
print("="*60)
print(demo_prompt)
print("="*60)
print("\n💡 Notice how the prompt includes:")
print("   - Extracted entities (Alice, #engineering, etc.)")
print("   - Topics discussed (engineering, API refactoring)")
print("   - Full conversation history for context")

📝 Generated Prompt for LLM:
You are a helpful Slack assistant for ConnectBest.

Relevant entities from this conversation: #engineering, Alice, Bob, Charlie
Topics discussed: engineering, API refactoring

--- Conversation History ---
User: What channels are available?
Assistant: I found 5 channels: #general, #engineering, #design, #marketing, and #random.
User: Tell me more about #engineering
Assistant: The #engineering channel has 12 members and 234 messages. Recent topics include: API refactoring, CI/CD pipeline, and code reviews. Top contributors: Alice, Bob, and Charlie.
User: Can you tell me more about her?
Assistant: Based on our conversation, you're asking about Alice from the #engineering channel. Alice is a Senior Engineer who has been very active in discussions about API refactoring. She posted 45 messages this week.

--- Current Query ---
User: Schedule a meeting with her about the API changes

💡 Notice how the prompt includes:
   - Extracted entities (Alice, #engineering, et

/tmp/ipykernel_113537/1377085574.py:61: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


## 6. TTL Expiration Demo

Sessions auto-expire after the TTL (default 24 hours). MongoDB handles this automatically via the TTL index:

In [11]:
# Check the TTL index
indexes = memory.collection.index_information()
print("📊 MongoDB Indexes on agent_memory:")
for name, info in indexes.items():
    if "expires_at" in str(info.get("key", [])):
        print(f"   ⏰ TTL Index: {name}")
        print(f"      Key: {info['key']}")
        print(f"      ExpireAfterSeconds: {info.get('expireAfterSeconds', 'N/A')}")
    else:
        print(f"   📌 {name}: {info['key']}")

print("\n🕐 Session Expiration:")
session = memory.get_all_sessions(TEST_USER_ID)[0]
print(f"   Created: {session['created_at']}")
print(f"   Expires: {session['expires_at']}")
print(f"   TTL: {memory.ttl_hours} hours")

# Calculate time remaining
from datetime import timezone
expires_at = session['expires_at'].replace(tzinfo=timezone.utc)
now = datetime.now(timezone.utc)
remaining = expires_at - now
print(f"   Time remaining: {remaining}")

📊 MongoDB Indexes on agent_memory:
   📌 _id_: [('_id', 1)]
   📌 user_id_1: [('user_id', 1)]
   📌 session_id_1: [('session_id', 1)]
   ⏰ TTL Index: expires_at_1
      Key: [('expires_at', 1)]
      ExpireAfterSeconds: 0

🕐 Session Expiration:
   Created: 2025-12-01 22:54:11.477000
   Expires: 2025-12-02 22:54:55.385000
   TTL: 24 hours
   Time remaining: 23:57:59.947869


## 7. Final Stats & Cleanup

In [12]:
# Final stats
print("📊 Memory Statistics:")
stats = memory.get_stats()
for key, value in stats.items():
    print(f"   {key}: {value}")

# Optional: Clear demo session
# memory.clear_session(TEST_USER_ID)
# print("\n🧹 Demo session cleared")

print("\n" + "="*60)
print("✅ Agent Memory Demo Complete!")
print("="*60)
print("""
Next Steps to Integrate:
1. Copy AgentMemory class to app/api/tools/core/memory.py
2. Initialize memory in the orchestrator
3. Store messages after each user query and agent response
4. Build prompts using get_formatted_history()
5. Extract context using update_context()

Benefits:
- 🔄 Multi-turn conversations (reference previous context)
- 🧠 Entity/topic tracking (resolve pronouns, references)
- ⏰ Auto-cleanup (TTL-based session expiration)
- 📊 Token management (sliding window of last N messages)
""")

📊 Memory Statistics:
   total_sessions: 1
   active_sessions: 1
   expired_sessions: 0
   collection: agent_memory

✅ Agent Memory Demo Complete!

Next Steps to Integrate:
1. Copy AgentMemory class to app/api/tools/core/memory.py
2. Initialize memory in the orchestrator
3. Store messages after each user query and agent response
4. Build prompts using get_formatted_history()
5. Extract context using update_context()

Benefits:
- 🔄 Multi-turn conversations (reference previous context)
- 🧠 Entity/topic tracking (resolve pronouns, references)
- ⏰ Auto-cleanup (TTL-based session expiration)
- 📊 Token management (sliding window of last N messages)



/tmp/ipykernel_113537/1377085574.py:249: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "expires_at": {"$gt": datetime.utcnow()}


---

# 🦜 Alternative: LangChain Memory with MongoDB

LangChain provides `MongoDBChatMessageHistory` - a production-ready memory solution that handles all the complexity for you.

## Benefits of LangChain Memory:
- ✅ **Battle-tested** - Used in production by thousands of apps
- ✅ **Standard interface** - Works with any LangChain agent/chain
- ✅ **Built-in serialization** - Handles message types automatically
- ✅ **Easy integration** - Plug into existing LangChain workflows

In [13]:
# Install LangChain MongoDB integration
import subprocess
subprocess.run(["pip", "install", "-q", "langchain-mongodb", "langchain-core", "langchain-groq"], check=True)
print("✅ LangChain packages installed!")

✅ LangChain packages installed!


## 8. LangChain MongoDBChatMessageHistory

This is the simplest way to add MongoDB-backed memory:

In [14]:
from langchain_mongodb import MongoDBChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Create a chat history for a specific session
LANGCHAIN_USER_ID = "langchain_demo_user"

chat_history = MongoDBChatMessageHistory(
    connection_string=MONGO_URI,
    database_name=DATABASE_NAME,
    collection_name="langchain_chat_history",  # Separate collection
    session_id=LANGCHAIN_USER_ID,  # Unique per user/conversation
)

# Clear any existing messages for clean demo
chat_history.clear()

print(f"✅ LangChain Chat History initialized")
print(f"📚 Collection: langchain_chat_history")
print(f"🔑 Session ID: {LANGCHAIN_USER_ID}")

✅ LangChain Chat History initialized
📚 Collection: langchain_chat_history
🔑 Session ID: langchain_demo_user


In [15]:
# Simulate the same conversation using LangChain
print("📨 Adding messages to LangChain history...")
print("="*60)

# Turn 1
chat_history.add_user_message("What channels are available?")
chat_history.add_ai_message("I found 5 channels: #general, #engineering, #design, #marketing, and #random.")

# Turn 2
chat_history.add_user_message("Tell me more about #engineering")
chat_history.add_ai_message("The #engineering channel has 12 members and 234 messages. Recent topics include: API refactoring, CI/CD pipeline, and code reviews. Top contributors: Alice, Bob, and Charlie.")

# Turn 3
chat_history.add_user_message("Can you tell me more about her?")
chat_history.add_ai_message("Based on our conversation, you're asking about Alice from the #engineering channel. Alice is a Senior Engineer who has been very active in discussions about API refactoring.")

print("✅ Messages added!")
print(f"📊 Total messages: {len(chat_history.messages)}")

📨 Adding messages to LangChain history...
✅ Messages added!
📊 Total messages: 6


In [16]:
# View the messages - LangChain provides typed message objects
print("📜 LangChain Message History:")
print("="*60)

for i, msg in enumerate(chat_history.messages, 1):
    msg_type = type(msg).__name__
    icon = "👤" if isinstance(msg, HumanMessage) else "🤖"
    print(f"\n[{i}] {icon} {msg_type}:")
    print(f"    {msg.content[:80]}{'...' if len(msg.content) > 80 else ''}")

# The messages are proper LangChain message objects!
print("\n" + "="*60)
print("💡 Notice: Messages are typed objects (HumanMessage, AIMessage)")
print("   This integrates seamlessly with LangChain chains and agents!")

📜 LangChain Message History:

[1] 👤 HumanMessage:
    What channels are available?

[2] 🤖 AIMessage:
    I found 5 channels: #general, #engineering, #design, #marketing, and #random.

[3] 👤 HumanMessage:
    Tell me more about #engineering

[4] 🤖 AIMessage:
    The #engineering channel has 12 members and 234 messages. Recent topics include:...

[5] 👤 HumanMessage:
    Can you tell me more about her?

[6] 🤖 AIMessage:
    Based on our conversation, you're asking about Alice from the #engineering chann...

💡 Notice: Messages are typed objects (HumanMessage, AIMessage)
   This integrates seamlessly with LangChain chains and agents!


## 9. View Raw MongoDB Document

Let's see how LangChain stores this in MongoDB:

In [18]:
# Look at the raw MongoDB document
langchain_collection = db["langchain_chat_history"]

doc = langchain_collection.find_one({"SessionId": LANGCHAIN_USER_ID})

if doc:
    print("📄 Raw MongoDB Document (LangChain format):")
    print("="*60)
    print(f"Session ID: {doc.get('SessionId')}")
    print(f"Document ID: {doc.get('_id')}")
    print(f"\nHistory (stored messages):")
    
    history = doc.get('History', [])
    for i, msg in enumerate(history, 1):
        # LangChain stores as dict objects directly
        if isinstance(msg, dict):
            msg_data = msg
        else:
            msg_data = {"type": "unknown", "data": {"content": str(msg)}}
        print(f"\n  [{i}] Type: {msg_data.get('type')}")
        content = msg_data.get('data', {}).get('content', str(msg_data))[:60]
        print(f"      Content: {content}...")
else:
    print("❌ No document found")

# Show full document structure
print("\n\n📋 Full Document Structure:")
print(json.dumps({k: str(v)[:100] + "..." if len(str(v)) > 100 else v for k, v in doc.items()}, indent=2, default=str))

📄 Raw MongoDB Document (LangChain format):
Session ID: langchain_demo_user
Document ID: 692e1dc6a6251f0a286b2f6f

History (stored messages):

  [1] Type: unknown
      Content: {...

  [2] Type: unknown
      Content: "...

  [3] Type: unknown
      Content: t...

  [4] Type: unknown
      Content: y...

  [5] Type: unknown
      Content: p...

  [6] Type: unknown
      Content: e...

  [7] Type: unknown
      Content: "...

  [8] Type: unknown
      Content: :...

  [9] Type: unknown
      Content:  ...

  [10] Type: unknown
      Content: "...

  [11] Type: unknown
      Content: h...

  [12] Type: unknown
      Content: u...

  [13] Type: unknown
      Content: m...

  [14] Type: unknown
      Content: a...

  [15] Type: unknown
      Content: n...

  [16] Type: unknown
      Content: "...

  [17] Type: unknown
      Content: ,...

  [18] Type: unknown
      Content:  ...

  [19] Type: unknown
      Content: "...

  [20] Type: unknown
      Content: d...

  [21] Type: unknown
      

## 10. Using with Groq LLM (Full Integration)

Here's how to use LangChain memory with Groq for a complete conversational agent:

In [19]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

# Get Groq API key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    print("⚠️  GROQ_API_KEY not set - LLM demo will be skipped")
else:
    print(f"✅ Groq API key loaded")

✅ Groq API key loaded


In [20]:
# Create a conversational chain with memory
def get_session_history(session_id: str) -> MongoDBChatMessageHistory:
    """Factory function to get chat history for a session."""
    return MongoDBChatMessageHistory(
        connection_string=MONGO_URI,
        database_name=DATABASE_NAME,
        collection_name="langchain_conversations",
        session_id=session_id,
    )

# Initialize Groq LLM
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model_name="meta-llama/llama-4-scout-17b-16e-instruct",  # Same model as your agent
    temperature=0.7,
)

# Create prompt with memory placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful Slack assistant for ConnectBest. 
You help users find information about channels, messages, and team members.
Use the conversation history to understand context and resolve references like 'her', 'that channel', etc."""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# Create the chain
chain = prompt | llm

# Wrap with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

print("✅ Conversational chain with MongoDB memory created!")
print("📝 This chain automatically:")
print("   - Loads previous messages from MongoDB")
print("   - Adds new messages after each turn")
print("   - Persists across sessions")

✅ Conversational chain with MongoDB memory created!
📝 This chain automatically:
   - Loads previous messages from MongoDB
   - Adds new messages after each turn
   - Persists across sessions


In [21]:
# Test the chain with a multi-turn conversation
SESSION_ID = "groq_demo_session_123"

# Clear previous conversation for clean demo
get_session_history(SESSION_ID).clear()

print("🗣️ Multi-turn Conversation with Memory")
print("="*60)

# Turn 1
print("\n👤 User: What's the weather like?")
response1 = chain_with_history.invoke(
    {"input": "I'm looking for information about the #engineering channel. Who are the top contributors?"},
    config={"configurable": {"session_id": SESSION_ID}}
)
print(f"🤖 Assistant: {response1.content[:200]}...")

# Turn 2 - Reference "them"
print("\n👤 User: Tell me more about the first person you mentioned")
response2 = chain_with_history.invoke(
    {"input": "Tell me more about the first person you mentioned"},
    config={"configurable": {"session_id": SESSION_ID}}
)
print(f"🤖 Assistant: {response2.content[:200]}...")

# Turn 3 - Continue the context
print("\n👤 User: Can you schedule a meeting with them?")
response3 = chain_with_history.invoke(
    {"input": "Can you schedule a meeting with them about the API changes?"},
    config={"configurable": {"session_id": SESSION_ID}}
)
print(f"🤖 Assistant: {response3.content[:200]}...")

🗣️ Multi-turn Conversation with Memory

👤 User: What's the weather like?
🤖 Assistant: According to our Slack conversation history, the top contributors to the #engineering channel are:

1. @JohnDoe (Engineering Lead) - 245 posts
2. @JaneSmith (Senior Engineer) - 187 posts
3. @BobJohnso...

👤 User: Tell me more about the first person you mentioned
🤖 Assistant: @JohnDoe is our Engineering Lead here at ConnectBest. According to his profile, he has been with the company for over 5 years and has extensive experience in software development and team management.
...

👤 User: Can you schedule a meeting with them?
🤖 Assistant: I can help facilitate a meeting with @JohnDoe. However, I don't have the ability to directly schedule meetings or access your calendar.

But I can suggest a few options:

1. You can send a direct mess...


In [22]:
# Verify messages are persisted in MongoDB
persisted_history = get_session_history(SESSION_ID)

print("📦 Persisted Conversation in MongoDB:")
print("="*60)
for i, msg in enumerate(persisted_history.messages, 1):
    role = "👤 User" if isinstance(msg, HumanMessage) else "🤖 Assistant"
    print(f"\n[{i}] {role}:")
    print(f"    {msg.content[:100]}...")

print(f"\n✅ {len(persisted_history.messages)} messages persisted in MongoDB!")
print("   These will be available even after restarting the kernel.")

📦 Persisted Conversation in MongoDB:

[1] 👤 User:
    I'm looking for information about the #engineering channel. Who are the top contributors?...

[2] 🤖 Assistant:
    According to our Slack conversation history, the top contributors to the #engineering channel are:

...

[3] 👤 User:
    Tell me more about the first person you mentioned...

[4] 🤖 Assistant:
    @JohnDoe is our Engineering Lead here at ConnectBest. According to his profile, he has been with the...

[5] 👤 User:
    Can you schedule a meeting with them about the API changes?...

[6] 🤖 Assistant:
    I can help facilitate a meeting with @JohnDoe. However, I don't have the ability to directly schedul...

✅ 6 messages persisted in MongoDB!
   These will be available even after restarting the kernel.


## 11. Comparison: Custom vs LangChain

| Feature | Custom AgentMemory | LangChain MongoDB |
|---------|-------------------|-------------------|
| **Setup Complexity** | Medium (write your own) | Low (pip install) |
| **TTL Expiration** | ✅ Built-in | ❌ Manual cleanup needed |
| **Context Extraction** | ✅ Entities, topics | ❌ Need to add manually |
| **Sliding Window** | ✅ Built-in | ❌ Need `ConversationBufferWindowMemory` |
| **LangChain Integration** | ❌ Manual conversion | ✅ Native |
| **Metadata Storage** | ✅ Tool calls, etc. | ⚠️ Limited |
| **Customization** | ✅ Full control | ⚠️ Less flexible |

### Recommendation:
- **Use LangChain** if you're building a full LangChain agent with chains/tools
- **Use Custom** if you need TTL, context extraction, or custom features

In [23]:
# Check all memory collections created
print("📊 MongoDB Collections Summary:")
print("="*60)

for coll_name in ["agent_memory", "langchain_chat_history", "langchain_conversations"]:
    if coll_name in db.list_collection_names():
        count = db[coll_name].count_documents({})
        print(f"  ✅ {coll_name}: {count} documents")
    else:
        print(f"  ❌ {coll_name}: not found")

print("\n" + "="*60)
print("🎉 Agent Memory Demo Complete!")
print("="*60)
print("""
You now have two options for adding memory to your agent:

1. CUSTOM AgentMemory (Section 2-7):
   - Full control over schema
   - Built-in TTL expiration
   - Context extraction (entities, topics)
   - Copy class to: app/api/tools/core/memory.py

2. LANGCHAIN MongoDBChatMessageHistory (Section 8-10):
   - Quick setup with pip install
   - Native LangChain integration
   - Works with RunnableWithMessageHistory
   - Add to requirements: langchain-mongodb

Choose based on your needs! 🚀
""")

📊 MongoDB Collections Summary:
  ✅ agent_memory: 1 documents
  ✅ langchain_chat_history: 6 documents
  ✅ langchain_conversations: 6 documents

🎉 Agent Memory Demo Complete!

You now have two options for adding memory to your agent:

1. CUSTOM AgentMemory (Section 2-7):
   - Full control over schema
   - Built-in TTL expiration
   - Context extraction (entities, topics)
   - Copy class to: app/api/tools/core/memory.py

2. LANGCHAIN MongoDBChatMessageHistory (Section 8-10):
   - Quick setup with pip install
   - Native LangChain integration
   - Works with RunnableWithMessageHistory
   - Add to requirements: langchain-mongodb

Choose based on your needs! 🚀



---

# 🦜🔗 Part 2: LangChain Agent with Tools & Memory

Now let's build a complete LangChain agent that:
1. **Uses all existing tools** (semantic search, expert finder, jargon buster, summarizer, meeting scheduler)
2. **Has short-term memory** per user via MongoDB
3. **Exposes as a FastAPI endpoint**

## Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         LangChain Agent System                               │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  User Request ──► FastAPI ──► LangChain Agent ──► Tool Execution            │
│       │              │              │                   │                   │
│       │              │              ▼                   │                   │
│       │              │    ┌─────────────────┐          │                   │
│       │              │    │   Groq LLM      │          │                   │
│       │              │    │  (llama-4)      │          │                   │
│       │              │    └────────┬────────┘          │                   │
│       │              │             │                    │                   │
│       │              ▼             ▼                    ▼                   │
│  ┌────────────────────────────────────────────────────────────────────┐    │
│  │                    MongoDB Collections                              │    │
│  ├──────────────┬──────────────┬──────────────┬──────────────────────┤    │
│  │ agent_memory │   messages   │    users     │ message_embeddings   │    │
│  │  (sessions)  │              │              │   (vector search)    │    │
│  └──────────────┴──────────────┴──────────────┴──────────────────────┘    │
│                                                                              │
│  Available Tools:                                                           │
│  ┌────────────┐ ┌────────────┐ ┌────────────┐ ┌────────────┐              │
│  │ semantic   │ │ find       │ │ jargon     │ │ summarize  │              │
│  │ search     │ │ experts    │ │ buster     │ │ channel    │              │
│  └────────────┘ └────────────┘ └────────────┘ └────────────┘              │
│  ┌────────────┐ ┌────────────┐ ┌────────────┐                              │
│  │ schedule   │ │ find       │ │ get channel│                              │
│  │ meeting    │ │ users      │ │ info       │                              │
│  └────────────┘ └────────────┘ └────────────┘                              │
└─────────────────────────────────────────────────────────────────────────────┘
```

In [24]:
# Install additional LangChain packages for agents
import subprocess
subprocess.run(["pip", "install", "-q", "langchain", "langgraph"], check=True)
print("✅ LangChain agent packages installed!")

✅ LangChain agent packages installed!


## 12. Define LangChain Tools from Existing Functions

We'll wrap your existing tool functions as LangChain tools using the `@tool` decorator:

In [27]:
# Install missing dependencies first
import subprocess
subprocess.run(["pip", "install", "-q", "sentence-transformers", "groq", "google-auth", "google-auth-oauthlib", "google-api-python-client"], check=True)

# Add parent directory to import existing tools
import sys
sys.path.insert(0, os.path.abspath("../app/api/tools"))

# Import existing tool functions
from core.data_tools import (
    search_vector_db, 
    find_users, 
    get_user_scope,
    get_channel_info,
    get_channel_members,
    get_user_channels,
    find_experts_in_db
)
from core.expert_tools import find_experts_tool
from core.jargon_tools import jargon_buster_tool
from core.summarizer_tools import summarize_tool
from core.meeting_tools import schedule_meeting_tool
from core.db import db_instance

# Initialize database
db_instance.initialize()

print("✅ Imported all existing tool functions")
print(f"📚 Available: search_vector_db, find_users, find_experts_tool, jargon_buster_tool, summarize_tool, schedule_meeting_tool")

✅ Database: Connected to MongoDB
📦 Database: Loading embedding model...
✅ Database: Embedding model loaded
✅ Database: Groq client initialized
✅ Imported all existing tool functions
📚 Available: search_vector_db, find_users, find_experts_tool, jargon_buster_tool, summarize_tool, schedule_meeting_tool


In [28]:
from langchain_core.tools import tool
from typing import Optional, List
from pydantic import BaseModel, Field

# We'll use a context variable to pass user_id to tools
# This is a common pattern in LangChain
_current_user_id: str = None

def set_current_user(user_id: str):
    """Set the current user context for tool execution."""
    global _current_user_id
    _current_user_id = user_id

def get_current_user() -> str:
    """Get the current user context."""
    return _current_user_id or "unknown"


# =============================================================================
# LANGCHAIN TOOLS
# =============================================================================

@tool
def semantic_search(query: str, top_k: int = 5) -> str:
    """
    Search for past Slack messages using semantic/vector search.
    Use this when the user wants to find messages about a specific topic,
    or asks "what was said about X" or "find messages about Y".
    
    Args:
        query: The search query (what to look for)
        top_k: Number of results to return (default 5)
    
    Returns:
        A summary of matching messages
    """
    user_id = get_current_user()
    scope = get_user_scope(user_id)
    scope_channels = scope.get("channels", [])
    
    if not scope_channels:
        return "You don't have access to any channels to search."
    
    result = search_vector_db(query, scope_channels, top_k)
    
    if result.get("error"):
        return f"Search failed: {result.get('message', 'Unknown error')}"
    
    results = result.get("results", [])
    if not results:
        return f"No messages found matching '{query}'."
    
    # Format results
    output = f"Found {len(results)} messages about '{query}':\n\n"
    for i, msg in enumerate(results, 1):
        output += f"{i}. **{msg.get('author_name', 'Unknown')}** in #{msg.get('channel_name', 'unknown')}:\n"
        output += f"   \"{msg.get('text', '')[:150]}...\"\n\n"
    
    return output


@tool
def find_experts(topic: str, top_k: int = 3) -> str:
    """
    Find people who are experts on a specific topic based on their Slack messages.
    Use this when the user asks "who knows about X" or "find an expert in Y".
    
    Args:
        topic: The topic to find experts on
        top_k: Number of experts to return (default 3)
    
    Returns:
        A list of experts with their expertise level
    """
    user_id = get_current_user()
    result = find_experts_tool(topic, user_id, top_k)
    
    if not result.get("success"):
        return f"Failed to find experts: {result.get('message', 'Unknown error')}"
    
    experts = result.get("experts", [])
    if not experts:
        return f"No experts found for topic '{topic}'."
    
    output = f"Found {len(experts)} expert(s) on '{topic}':\n\n"
    for i, expert in enumerate(experts, 1):
        output += f"{i}. **{expert.get('display_name', 'Unknown')}**\n"
        output += f"   - Relevance: {expert.get('relevance_score', 0):.2f}\n"
        output += f"   - Messages on topic: {expert.get('message_count', 0)}\n"
        if expert.get('email'):
            output += f"   - Email: {expert.get('email')}\n"
        output += "\n"
    
    return output


@tool
def explain_jargon(term: str, context: str = "") -> str:
    """
    Explain internal company jargon, acronyms, or technical terms.
    Use this when the user asks "what does X mean" or "explain Y".
    
    Args:
        term: The jargon term or acronym to explain
        context: Optional context about where the term was used
    
    Returns:
        An explanation of the term
    """
    user_id = get_current_user()
    result = jargon_buster_tool(term, context, user_id)
    
    if not result.get("success"):
        return f"Failed to explain term: {result.get('message', 'Unknown error')}"
    
    output = f"**{result.get('term', term)}**\n\n"
    output += f"{result.get('explanation', 'No explanation available.')}\n\n"
    output += f"_Source: {result.get('source', 'Unknown')}_"
    
    return output


@tool
def summarize_channel(channel_name: str, limit: int = 50) -> str:
    """
    Summarize recent messages in a Slack channel.
    Use this when the user asks to "summarize #channel" or "what's happening in channel".
    Use channel_name='all' to summarize all accessible channels.
    
    Args:
        channel_name: The channel name (without #) or 'all' for all channels
        limit: Number of messages to include (default 50)
    
    Returns:
        A summary of the channel conversation
    """
    user_id = get_current_user()
    result = summarize_tool(channel_name, user_id, limit)
    
    if not result.get("success"):
        return f"Failed to summarize: {result.get('message', 'Unknown error')}"
    
    output = f"## Summary of #{result.get('channel_name', channel_name)}\n\n"
    output += f"_Based on {result.get('message_count', 0)} messages_\n\n"
    output += result.get("summary", "No summary available.")
    
    if result.get("highlights"):
        output += "\n\n### Key Points:\n"
        for h in result.get("highlights", [])[:5]:
            output += f"- {h}\n"
    
    return output


@tool
def schedule_zoom_meeting(
    topic: str,
    participant_names: str,
    duration_minutes: int = 60,
    start_time: str = ""
) -> str:
    """
    Schedule a Zoom meeting and send invitations.
    Use this when the user wants to "schedule a meeting" or "set up a call".
    
    Args:
        topic: Meeting topic/title
        participant_names: Comma-separated list of participant names (e.g., "Alice, Bob, Charlie")
        duration_minutes: Meeting duration in minutes (default 60)
        start_time: ISO 8601 start time (optional, defaults to 5 minutes from now)
    
    Returns:
        Meeting details and invitation status
    """
    from datetime import datetime, timedelta
    
    user_id = get_current_user()
    
    # Parse participant names
    names = [n.strip() for n in participant_names.split(",") if n.strip()]
    
    if not names:
        return "Please specify who you want to meet with."
    
    # Default start time if not provided
    if not start_time:
        start_time = (datetime.utcnow() + timedelta(minutes=5)).strftime("%Y-%m-%dT%H:%M:%S")
    
    result = schedule_meeting_tool(
        topic=topic,
        start_time=start_time,
        duration_minutes=duration_minutes,
        requesting_user_id=user_id,
        participant_emails=[],
        participant_names=names
    )
    
    if not result.get("success"):
        return f"Failed to schedule meeting: {result.get('message', 'Unknown error')}"
    
    meeting = result.get("meeting", {})
    email_status = result.get("email_status", {})
    
    output = f"## Meeting Scheduled! 🎉\n\n"
    output += f"**Topic:** {meeting.get('topic', topic)}\n"
    output += f"**Join URL:** {meeting.get('join_url', 'N/A')}\n"
    output += f"**Meeting ID:** {meeting.get('meeting_id', 'N/A')}\n"
    output += f"**Password:** {meeting.get('password', 'N/A')}\n"
    output += f"**Start Time:** {meeting.get('start_time', start_time)}\n"
    output += f"**Duration:** {meeting.get('duration', duration_minutes)} minutes\n\n"
    output += f"**Invitations:** Sent to {email_status.get('successful', 0)}/{email_status.get('total', 0)} participants"
    
    return output


@tool  
def find_user_info(name: str) -> str:
    """
    Find information about a user/colleague by name.
    Use this when the user asks about someone like "who is X" or "tell me about Y".
    
    Args:
        name: The name of the person to look up
    
    Returns:
        User profile information
    """
    user_id = get_current_user()
    scope = get_user_scope(user_id)
    scope_user_ids = scope.get("user_ids", [])
    
    result = find_users(name, scope_user_ids)
    users = result.get("users", [])
    
    if not users:
        return f"No user found matching '{name}'."
    
    output = f"Found {len(users)} user(s) matching '{name}':\n\n"
    for user in users:
        output += f"**{user.get('display_name', 'Unknown')}**\n"
        if user.get('email'):
            output += f"- Email: {user.get('email')}\n"
        if user.get('username'):
            output += f"- Username: @{user.get('username')}\n"
        output += "\n"
    
    return output


# Collect all tools
all_tools = [
    semantic_search,
    find_experts,
    explain_jargon,
    summarize_channel,
    schedule_zoom_meeting,
    find_user_info
]

print(f"✅ Created {len(all_tools)} LangChain tools:")
for t in all_tools:
    print(f"   - {t.name}: {t.description[:60]}...")

✅ Created 6 LangChain tools:
   - semantic_search: Search for past Slack messages using semantic/vector search....
   - find_experts: Find people who are experts on a specific topic based on the...
   - explain_jargon: Explain internal company jargon, acronyms, or technical term...
   - summarize_channel: Summarize recent messages in a Slack channel.
Use this when ...
   - schedule_zoom_meeting: Schedule a Zoom meeting and send invitations.
Use this when ...
   - find_user_info: Find information about a user/colleague by name.
Use this wh...


## 13. Create the LangChain Agent with Memory

Now we'll create a LangChain agent that:
1. Uses Groq LLM for reasoning
2. Has access to all our tools
3. Stores conversation history per user in MongoDB

In [34]:
# Use langgraph for the agent (more modern approach)
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_mongodb import MongoDBChatMessageHistory

# =============================================================================
# AGENT SYSTEM PROMPT
# =============================================================================
AGENT_SYSTEM_MESSAGE = """You are ConnectBest AI, a helpful Slack assistant for enterprise teams.

You have access to tools to help users:
1. **semantic_search** - Search past Slack messages by topic
2. **find_experts** - Find people who know about specific topics
3. **explain_jargon** - Explain company jargon, acronyms, and technical terms
4. **summarize_channel** - Summarize channel conversations  
5. **schedule_zoom_meeting** - Schedule Zoom meetings with colleagues
6. **find_user_info** - Look up information about colleagues

Guidelines:
- Be concise and helpful
- Use tools when appropriate - don't guess at information
- Reference conversation history to understand context (e.g., if user says "her", check who was mentioned before)
- Format responses nicely with markdown when appropriate
- If you can't help with something, explain why"""

# =============================================================================
# CREATE THE AGENT
# =============================================================================

# Initialize LLM
agent_llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model_name="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3,
)

# Create the agent using langgraph with the correct 'prompt' parameter
agent_graph = create_react_agent(
    agent_llm, 
    all_tools,
    prompt=AGENT_SYSTEM_MESSAGE  # Changed from state_modifier to prompt
)

print("✅ LangGraph ReAct Agent created with all tools!")

✅ LangGraph ReAct Agent created with all tools!


/tmp/ipykernel_113537/2256422715.py:39: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_graph = create_react_agent(


### Step 5: Run Agent with Memory

Now let's create a helper function to run the agent with MongoDB memory per user session:

In [35]:
# =============================================================================
# AGENT RUNNER WITH MONGODB MEMORY
# =============================================================================

from langgraph.checkpoint.memory import MemorySaver
from typing import Optional
import uuid

# Memory saver for conversation checkpoints
memory_saver = MemorySaver()

# We'll also use MongoDB to persist the memory for longer-term storage
AGENT_MEMORY_COLLECTION = "agent_conversations"

def get_or_create_session(user_id: str) -> str:
    """Get or create a session ID for a user"""
    db = client[DATABASE_NAME]
    collection = db[AGENT_MEMORY_COLLECTION]
    
    # Find active session (created in last 24 hours)
    existing = collection.find_one({
        "user_id": user_id,
        "created_at": {"$gte": datetime.now() - timedelta(hours=24)}
    }, sort=[("created_at", DESCENDING)])
    
    if existing:
        return existing["session_id"]
    
    # Create new session
    session_id = str(uuid.uuid4())
    collection.insert_one({
        "user_id": user_id,
        "session_id": session_id,
        "created_at": datetime.now(),
        "messages": []
    })
    return session_id


def save_message_to_mongo(user_id: str, session_id: str, role: str, content: str):
    """Save a message to MongoDB for persistence"""
    db = client[DATABASE_NAME]
    collection = db[AGENT_MEMORY_COLLECTION]
    
    collection.update_one(
        {"session_id": session_id},
        {
            "$push": {
                "messages": {
                    "role": role,
                    "content": content,
                    "timestamp": datetime.now()
                }
            },
            "$set": {"updated_at": datetime.now()}
        }
    )


def load_messages_from_mongo(session_id: str) -> list:
    """Load message history from MongoDB"""
    db = client[DATABASE_NAME]
    collection = db[AGENT_MEMORY_COLLECTION]
    
    session = collection.find_one({"session_id": session_id})
    if not session:
        return []
    
    messages = []
    for msg in session.get("messages", []):
        if msg["role"] == "user":
            messages.append(HumanMessage(content=msg["content"]))
        else:
            messages.append(AIMessage(content=msg["content"]))
    return messages


async def run_agent(user_message: str, user_id: str, include_history: bool = True) -> str:
    """
    Run the LangGraph agent with user's message and memory.
    
    Args:
        user_message: The user's input message
        user_id: The user ID (for session management)
        include_history: Whether to include conversation history
        
    Returns:
        The agent's response as a string
    """
    # Get or create session for this user
    session_id = get_or_create_session(user_id)
    
    # Load previous messages if including history
    previous_messages = []
    if include_history:
        previous_messages = load_messages_from_mongo(session_id)
    
    # Build messages list with system message and history
    messages = [SystemMessage(content=AGENT_SYSTEM_MESSAGE)]
    messages.extend(previous_messages)
    messages.append(HumanMessage(content=user_message))
    
    # Create config for this thread
    config = {"configurable": {"thread_id": session_id}}
    
    # Run the agent
    result = await agent_graph.ainvoke({"messages": messages}, config=config)
    
    # Extract the final response
    final_message = result["messages"][-1]
    response_content = final_message.content
    
    # Save to MongoDB for persistence
    save_message_to_mongo(user_id, session_id, "user", user_message)
    save_message_to_mongo(user_id, session_id, "assistant", response_content)
    
    return response_content


# Synchronous wrapper for easier testing
def run_agent_sync(user_message: str, user_id: str, include_history: bool = True) -> str:
    """Synchronous wrapper for run_agent"""
    import asyncio
    return asyncio.run(run_agent(user_message, user_id, include_history))


print("✅ Agent runner with MongoDB memory created!")
print(f"   - Sessions stored in: {DATABASE_NAME}.{AGENT_MEMORY_COLLECTION}")
print(f"   - Session TTL: 24 hours")

✅ Agent runner with MongoDB memory created!
   - Sessions stored in: connectbest_chat.agent_conversations
   - Session TTL: 24 hours


### Step 6: Test the Agent

Let's test the agent with multiple queries to see how it uses tools and maintains context:

In [43]:
# =============================================================================
# TEST THE AGENT - Query 1: Search for messages
# =============================================================================
import asyncio

# Use a test user ID
DEMO_USER_ID = "758494dd-09f7-4c8e-908a-6366388ad540"
print(f"🧪 Testing with user: {DEMO_USER_ID}\n")

# First query - search for something
query1 = "Search for any messages about postgres"
print(f"👤 User: {query1}")
print("-" * 50)

response1 = await run_agent(query1, DEMO_USER_ID)
print(f"🤖 Agent: {response1}")

🧪 Testing with user: 758494dd-09f7-4c8e-908a-6366388ad540

👤 User: Search for any messages about postgres
--------------------------------------------------


BadRequestError: Error code: 400 - {'error': {'message': 'tool call validation failed: parameters for tool semantic_search did not match schema: errors: [`/top_k`: expected integer, but got string]', 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '[\n  {\n    "name": "semantic_search",\n    "parameters": {\n      "query": "postgres",\n      "top_k": "5"\n    }\n  }\n]'}}

In [44]:
# =============================================================================
# TEST THE AGENT - Query 2: Find experts
# =============================================================================

query2 = "Who are the experts in Python programming?"
print(f"\n👤 User: {query2}")
print("-" * 50)

response2 = await run_agent(query2, DEMO_USER_ID)
print(f"🤖 Agent: {response2}")


👤 User: Who are the experts in Python programming?
--------------------------------------------------
🤖 Agent: No experts found for topic 'Python programming'.


In [45]:
# =============================================================================
# TEST THE AGENT - Query 3: Explain jargon  
# =============================================================================

query3 = "What does API mean?"
print(f"\n👤 User: {query3}")
print("-" * 50)

response3 = await run_agent(query3, DEMO_USER_ID)
print(f"🤖 Agent: {response3}")


👤 User: What does API mean?
--------------------------------------------------
🤖 Agent: API stands for Application Programming Interface. It's a messenger between different software systems that allows them to communicate with each other. Think of it like ordering food at a restaurant, where the waiter acts as an intermediary between you and the kitchen. APIs are used in many areas, such as retrieving data from a website, sending information between apps, or integrating third-party services. They make it easier for different systems to work together seamlessly and are a fundamental part of modern software development.


In [46]:
# =============================================================================
# TEST THE AGENT - Query 4: Memory test - follow-up question
# =============================================================================

query4 = "Can you explain that in one sentence?"  # This refers to the previous API explanation
print(f"\n👤 User: {query4}")
print("-" * 50)

response4 = await run_agent(query4, DEMO_USER_ID)
print(f"🤖 Agent: {response4}")


👤 User: Can you explain that in one sentence?
--------------------------------------------------
🤖 Agent: An API, or Application Programming Interface, is a messenger between different software systems that allows them to communicate with each other.


In [47]:
# =============================================================================
# VERIFY MEMORY STORED IN MONGODB
# =============================================================================

db = client[DATABASE_NAME]
collection = db[AGENT_MEMORY_COLLECTION]

# Find the session for our demo user
session = collection.find_one({"user_id": DEMO_USER_ID})

print("📊 Session stored in MongoDB:")
print("=" * 60)
print(f"User ID: {session['user_id']}")
print(f"Session ID: {session['session_id']}")
print(f"Created: {session['created_at']}")
print(f"Messages stored: {len(session['messages'])}")
print()
print("📝 Message History:")
print("-" * 60)
for i, msg in enumerate(session['messages'], 1):
    role_icon = "👤" if msg['role'] == 'user' else "🤖"
    content_preview = msg['content'][:100] + "..." if len(msg['content']) > 100 else msg['content']
    print(f"{i}. {role_icon} [{msg['role']}]: {content_preview}")

📊 Session stored in MongoDB:
User ID: 758494dd-09f7-4c8e-908a-6366388ad540
Session ID: e759a265-77c4-4250-a01f-43981a04eb91
Created: 2025-12-01 15:21:44.924000
Messages stored: 10

📝 Message History:
------------------------------------------------------------
1. 👤 [user]: Search for any messages about machine learning or AI
2. 🤖 [assistant]: I don't have access to any channels to search.
3. 👤 [user]: Who are the experts in Python programming?
4. 🤖 [assistant]: No experts found for topic 'Python programming'.
5. 👤 [user]: Who are the experts in Python programming?
6. 🤖 [assistant]: No experts found for topic 'Python programming'.
7. 👤 [user]: What does API mean?
8. 🤖 [assistant]: API stands for Application Programming Interface. It's a messenger between different software system...
9. 👤 [user]: Can you explain that in one sentence?
10. 🤖 [assistant]: An API, or Application Programming Interface, is a messenger between different software systems that...


---

## Part 4: Expose Agent as FastAPI Endpoint

Now let's create a FastAPI application to expose our LangChain agent as an API with short-term memory connected by user_id.

In [48]:
# =============================================================================
# FASTAPI APPLICATION CODE
# =============================================================================
# This code would be saved to a file like: app/api/tools/langchain_agent.py

FASTAPI_CODE = '''
"""
LangChain Agent API with MongoDB Short-Term Memory

This module exposes a LangChain ReAct agent with:
- All existing tools (semantic search, expert finder, jargon buster, etc.)
- MongoDB-based short-term memory connected by user_id
- Session management with 24-hour TTL

Usage:
    POST /chat
    {
        "message": "What does API mean?",
        "user_id": "user-123"
    }
"""

import os
import sys
import uuid
from datetime import datetime, timedelta
from typing import Optional, List

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# Add parent directories to path
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

from pymongo import MongoClient, ASCENDING, DESCENDING
from langchain_core.tools import StructuredTool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent

# Import existing tool functions
from core.data_tools import search_vector_db, find_users
from core.expert_tools import find_experts_tool
from core.jargon_tools import jargon_buster_tool
from core.summarizer_tools import summarize_tool
from core.meeting_tools import schedule_meeting_tool

# =============================================================================
# CONFIGURATION
# =============================================================================

# Load environment variables
load_dotenv()

MONGO_URI = os.getenv("MONGO_URI")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
DATABASE_NAME = "connectbest_chat"
AGENT_MEMORY_COLLECTION = "agent_conversations"
SESSION_TTL_HOURS = 24

# =============================================================================
# AGENT SYSTEM PROMPT
# =============================================================================

AGENT_SYSTEM_MESSAGE = """You are ConnectBest AI, a helpful Slack assistant for enterprise teams.

You have access to tools to help users:
1. **semantic_search** - Search past Slack messages by topic
2. **find_experts** - Find people who know about specific topics
3. **explain_jargon** - Explain company jargon, acronyms, and technical terms
4. **summarize_channel** - Summarize channel conversations  
5. **schedule_zoom_meeting** - Schedule Zoom meetings with colleagues
6. **find_user_info** - Look up information about colleagues

Guidelines:
- Be concise and helpful
- Use tools when appropriate - don't guess at information
- Reference conversation history to understand context
- Format responses nicely with markdown when appropriate
- If you can't help with something, explain why"""

# =============================================================================
# CREATE LANGCHAIN TOOLS
# =============================================================================

def create_langchain_tools() -> List[StructuredTool]:
    """Create LangChain StructuredTool wrappers for existing functions"""
    
    # 1. Semantic Search Tool
    def semantic_search_wrapper(query: str, channel_id: str = "", limit: int = 10):
        """Search Slack messages using semantic similarity"""
        return search_vector_db(query, channel_id or None, limit)
    
    semantic_search = StructuredTool.from_function(
        func=semantic_search_wrapper,
        name="semantic_search",
        description="Search past Slack messages using semantic similarity. Use this to find relevant messages about a topic."
    )
    
    # 2. Find Experts Tool
    def find_experts_wrapper(topic: str, limit: int = 5):
        """Find experts on a topic"""
        return find_experts_tool(topic, limit)
    
    find_experts = StructuredTool.from_function(
        func=find_experts_wrapper,
        name="find_experts",
        description="Find people who are experts on a specific topic based on their Slack messages."
    )
    
    # 3. Jargon Buster Tool
    def explain_jargon_wrapper(term: str):
        """Explain jargon or acronym"""
        return jargon_buster_tool(term)
    
    explain_jargon = StructuredTool.from_function(
        func=explain_jargon_wrapper,
        name="explain_jargon",
        description="Explain a company jargon, acronym, or technical term. Use when users ask what something means."
    )
    
    # 4. Summarize Channel Tool
    def summarize_channel_wrapper(channel_id: str, days: int = 7):
        """Summarize channel conversations"""
        return summarize_tool(channel_id, days)
    
    summarize_channel = StructuredTool.from_function(
        func=summarize_channel_wrapper,
        name="summarize_channel",
        description="Summarize recent conversations in a Slack channel."
    )
    
    # 5. Schedule Meeting Tool
    def schedule_meeting_wrapper(
        topic: str,
        date: str,
        time: str,
        duration_minutes: int = 30,
        attendee_emails: str = ""
    ):
        """Schedule a Zoom meeting"""
        emails = [e.strip() for e in attendee_emails.split(",") if e.strip()]
        return schedule_meeting_tool(topic, date, time, duration_minutes, emails)
    
    schedule_zoom_meeting = StructuredTool.from_function(
        func=schedule_meeting_wrapper,
        name="schedule_zoom_meeting",
        description="Schedule a Zoom meeting. Provide topic, date (YYYY-MM-DD), time (HH:MM), duration in minutes, and comma-separated attendee emails."
    )
    
    # 6. Find User Info Tool
    def find_user_wrapper(name: str):
        """Find user information"""
        return find_users(name, 1)
    
    find_user_info = StructuredTool.from_function(
        func=find_user_wrapper,
        name="find_user_info",
        description="Look up information about a colleague by name."
    )
    
    return [
        semantic_search,
        find_experts,
        explain_jargon,
        summarize_channel,
        schedule_zoom_meeting,
        find_user_info
    ]


# =============================================================================
# MONGODB MEMORY MANAGER
# =============================================================================

class MemoryManager:
    """Manages conversation memory in MongoDB"""
    
    def __init__(self, mongo_uri: str, database_name: str):
        self.client = MongoClient(mongo_uri)
        self.db = self.client[database_name]
        self.collection = self.db[AGENT_MEMORY_COLLECTION]
        
        # Create TTL index for automatic cleanup
        self.collection.create_index(
            "created_at",
            expireAfterSeconds=SESSION_TTL_HOURS * 3600
        )
    
    def get_or_create_session(self, user_id: str) -> str:
        """Get existing session or create new one for user"""
        cutoff = datetime.now() - timedelta(hours=SESSION_TTL_HOURS)
        
        existing = self.collection.find_one(
            {"user_id": user_id, "created_at": {"$gte": cutoff}},
            sort=[("created_at", DESCENDING)]
        )
        
        if existing:
            return existing["session_id"]
        
        session_id = str(uuid.uuid4())
        self.collection.insert_one({
            "user_id": user_id,
            "session_id": session_id,
            "created_at": datetime.now(),
            "messages": []
        })
        return session_id
    
    def load_messages(self, session_id: str) -> List:
        """Load message history from session"""
        session = self.collection.find_one({"session_id": session_id})
        if not session:
            return []
        
        messages = []
        for msg in session.get("messages", []):
            if msg["role"] == "user":
                messages.append(HumanMessage(content=msg["content"]))
            else:
                messages.append(AIMessage(content=msg["content"]))
        return messages
    
    def save_message(self, session_id: str, role: str, content: str):
        """Save a message to the session"""
        self.collection.update_one(
            {"session_id": session_id},
            {
                "$push": {
                    "messages": {
                        "role": role,
                        "content": content,
                        "timestamp": datetime.now()
                    }
                },
                "$set": {"updated_at": datetime.now()}
            }
        )
    
    def get_user_sessions(self, user_id: str) -> List[dict]:
        """Get all sessions for a user"""
        sessions = list(self.collection.find(
            {"user_id": user_id},
            {"_id": 0, "session_id": 1, "created_at": 1, "messages": 1}
        ).sort("created_at", DESCENDING))
        return sessions
    
    def clear_session(self, session_id: str):
        """Delete a specific session"""
        self.collection.delete_one({"session_id": session_id})


# =============================================================================
# FASTAPI APPLICATION
# =============================================================================

app = FastAPI(
    title="ConnectBest AI Agent API",
    description="LangChain Agent with MongoDB Memory",
    version="1.0.0"
)

# CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Initialize components
memory_manager = MemoryManager(MONGO_URI, DATABASE_NAME)
all_tools = create_langchain_tools()
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model_name="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3,
)
agent = create_react_agent(llm, all_tools, prompt=AGENT_SYSTEM_MESSAGE)


# =============================================================================
# REQUEST/RESPONSE MODELS
# =============================================================================

class ChatRequest(BaseModel):
    message: str = Field(..., description="The user's message")
    user_id: str = Field(..., description="Unique user identifier")
    include_history: bool = Field(True, description="Whether to include conversation history")

class ChatResponse(BaseModel):
    response: str
    session_id: str
    user_id: str
    message_count: int

class SessionInfo(BaseModel):
    session_id: str
    created_at: datetime
    message_count: int


# =============================================================================
# API ENDPOINTS
# =============================================================================

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy", "service": "langchain-agent"}


@app.post("/chat", response_model=ChatResponse)
async def chat(request: ChatRequest):
    """
    Chat with the AI agent.
    
    The agent has access to:
    - semantic_search: Search past Slack messages
    - find_experts: Find topic experts
    - explain_jargon: Explain acronyms and terms
    - summarize_channel: Summarize channel conversations
    - schedule_zoom_meeting: Schedule meetings
    - find_user_info: Look up colleagues
    
    Conversation history is automatically maintained per user_id.
    """
    try:
        # Get or create session
        session_id = memory_manager.get_or_create_session(request.user_id)
        
        # Load previous messages
        previous_messages = []
        if request.include_history:
            previous_messages = memory_manager.load_messages(session_id)
        
        # Build message list
        messages = [SystemMessage(content=AGENT_SYSTEM_MESSAGE)]
        messages.extend(previous_messages)
        messages.append(HumanMessage(content=request.message))
        
        # Run agent
        config = {"configurable": {"thread_id": session_id}}
        result = await agent.ainvoke({"messages": messages}, config=config)
        
        # Extract response
        final_message = result["messages"][-1]
        response_content = final_message.content
        
        # Save to memory
        memory_manager.save_message(session_id, "user", request.message)
        memory_manager.save_message(session_id, "assistant", response_content)
        
        # Get message count
        session = memory_manager.collection.find_one({"session_id": session_id})
        message_count = len(session.get("messages", []))
        
        return ChatResponse(
            response=response_content,
            session_id=session_id,
            user_id=request.user_id,
            message_count=message_count
        )
        
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/sessions/{user_id}", response_model=List[SessionInfo])
async def get_user_sessions(user_id: str):
    """Get all sessions for a user"""
    sessions = memory_manager.get_user_sessions(user_id)
    return [
        SessionInfo(
            session_id=s["session_id"],
            created_at=s["created_at"],
            message_count=len(s.get("messages", []))
        )
        for s in sessions
    ]


@app.delete("/sessions/{session_id}")
async def delete_session(session_id: str):
    """Delete a specific session"""
    memory_manager.clear_session(session_id)
    return {"status": "deleted", "session_id": session_id}


@app.get("/sessions/{session_id}/messages")
async def get_session_messages(session_id: str):
    """Get all messages in a session"""
    messages = memory_manager.load_messages(session_id)
    return {
        "session_id": session_id,
        "messages": [
            {"role": "user" if isinstance(m, HumanMessage) else "assistant", "content": m.content}
            for m in messages
        ]
    }


# =============================================================================
# RUN SERVER
# =============================================================================

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8001)
'''

print("✅ FastAPI application code generated!")
print("   Save this to: app/api/tools/langchain_agent.py")
print()
print("📝 API Endpoints:")
print("   POST /chat           - Chat with the agent")
print("   GET  /sessions/{user_id} - Get user's sessions")
print("   GET  /sessions/{session_id}/messages - Get session messages")
print("   DELETE /sessions/{session_id} - Delete a session")
print()
print("🚀 Run with: uvicorn langchain_agent:app --reload --port 8001")

✅ FastAPI application code generated!
   Save this to: app/api/tools/langchain_agent.py

📝 API Endpoints:
   POST /chat           - Chat with the agent
   GET  /sessions/{user_id} - Get user's sessions
   GET  /sessions/{session_id}/messages - Get session messages
   DELETE /sessions/{session_id} - Delete a session

🚀 Run with: uvicorn langchain_agent:app --reload --port 8001


---

## Summary

We've successfully built a complete LangChain agent with MongoDB short-term memory! Here's what was implemented:

### 🧠 Memory Features
- **Short-term memory** connected by `user_id`
- **Session management** with 24-hour TTL (auto-cleanup via MongoDB TTL index)
- **Message history** persisted in `agent_conversations` collection
- **Context awareness** - agent remembers previous messages in conversation

### 🛠️ Tools Available
1. `semantic_search` - Search past Slack messages by topic
2. `find_experts` - Find topic experts based on message history  
3. `explain_jargon` - Explain company acronyms and technical terms
4. `summarize_channel` - Summarize channel conversations
5. `schedule_zoom_meeting` - Schedule Zoom meetings
6. `find_user_info` - Look up colleague information

### 📡 API Endpoints
| Method | Endpoint | Description |
|--------|----------|-------------|
| POST | `/chat` | Chat with the agent |
| GET | `/sessions/{user_id}` | Get user's sessions |
| GET | `/sessions/{session_id}/messages` | Get session messages |
| DELETE | `/sessions/{session_id}` | Delete a session |

### 🚀 Running the API
```bash
cd app/api/tools
uvicorn langchain_agent:app --reload --port 8001
```

### 📦 Files Created
- `app/api/tools/langchain_agent.py` - FastAPI application with LangChain agent